## READ ME

Name: Kim, Minjun

Student ID: 20195024

Instructions: Please run the cells one by one. This is because the variable used in the previous cell is also used in the later cell.

***
***
### **Q7**

In [2]:
import numpy as np
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)

In [3]:
Boston = load_data("Boston")
Boston.tail(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,6.48,22.0
505,0.04741,0.0,11.93,0,0.573,6.030,80.8,2.5050,1,273,21.0,7.88,11.9


***
#### (a) Let's compute $\hat{\mu}$, an estimate for the population mean of medv data

In [4]:
medv_mean = Boston['medv'].mean()
print(f"An Estimate for the Population Mean of medv data: {medv_mean:.5f}")

An Estimate for the Population Mean of medv data: 22.53281


***
#### (b) Let's compute an estimate of the standard error of $\hat{\mu}$,which is $SE(\overline{X})=\frac{s}{\sqrt[]{n}}$

The standard error reflect how accurate an estimate of the mean is.  
To be more specific, a smaller standard error indicates that the sample mean is closer to the population mean.  
In this case, a value of 0.40886 so we can that the estimate of the mean is relatively accurate.

In [5]:
medv_se = Boston['medv'].std() / np.sqrt(Boston['medv'].count())
print(f"An Estimate of the Standard Error of mu hat: {medv_se:.5f}")

An Estimate of the Standard Error of mu hat: 0.40886


***
#### (c) Let's estimate the standard error of $\hat{\mu}$ using bootstrap


First, this generates bootstrap samples from the original data D, the Boston data.  
And I applied a function sample_mean to each sample, which returns mean of random samples.  
Finally, the boot_SE returns an estimate of standard error of mu hat by  calculating the standard error from the results.


So, an estimate of standard error of $\hat{\mu}$ by using boostrap is bigger than those gained from (b).  
This is because the bootstrap make samples randomly from the original data, which can lead to additional variability.

In [7]:
def sample_mean(D, idx):
    return D['medv'].loc[idx].mean()

def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

medv_se_bootstrap = boot_SE(sample_mean, Boston, n=None, B=1000, seed=0)


print(f"An Estimate of the Standard Error of mu hat by using Bootstrap: {medv_se_bootstrap:.5f}")
print(f"An Estimate of the Standard Error of mu hat from (b): {medv_se:.5f}")

An Estimate of the Standard Error of mu hat by using Bootstrap: 0.41253
An Estimate of the Standard Error of mu hat from (b): 0.40886


***
#### (d) Let's provide 95% confidence interval for $\hat{\mu}$, the mean of medv.

The confidence intervals from both methods are quite alike, showing that the bootstrap method might be a good way to estimate confidence intervals for this data.  
Also, because both intervals include $\hat{\mu}=22.53281$, an estimate for the population mean of medv, it suggests that those intervals are meaningful.

In [11]:
#95% confidence interval
def bootstrap_mean(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    means = []
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        means.append(func(D, idx))
    return np.mean(means)

medv_mean_bootstrap = bootstrap_mean(sample_mean, Boston, n=None, B=1000, seed=0)

conf_interval_bootstrap = [medv_mean_bootstrap - 2*medv_se_bootstrap, medv_mean_bootstrap+ 2*medv_se_bootstrap]
conf_interval_by_std = [medv_mean - 2*medv_se, medv_mean + 2*medv_se]

print(f"95% Confidence Interval by using Bootstrap: [{conf_interval_bootstrap[0]:.5f}, {conf_interval_bootstrap[1]:.5f}]")
print(f"95% Confidence Interval by using Boston['medv'].std(): [{conf_interval_by_std[0]:.5f}, {conf_interval_by_std[1]:.5f}]")

95% Confidence Interval by using Bootstrap: [21.72403, 23.37417]
95% Confidence Interval by using Boston['medv'].std(): [21.71508, 23.35053]


***
#### (e) Let's estimate $\hat\mu_{med}$ for the median value of medv in the population

In [98]:
medv_med = Boston['medv'].median()
print(f"An estimate of the median value of medv data: {medv_med:.5f}")

An estimate of the median value of medv data: 21.20000


***
#### (f) Let's estimate the standard error of $\hat\mu_{med}$ using boostrap

Since I already made boot_SE function previously for (c), all I need to do is just making a new definition of sample_median function that returns the median value of random samples.  

I found that an estimate of the standard error of median $\hat\mu$ by using Bootstrap is 0.36945, which is significantly small.  
This means that the sample median is likely near the median of the population.

In [111]:
def sample_median(D, idx):
    return D['medv'].loc[idx].median()

# Using boot_SE to estimate the standard error of the median.
medv_median_se_bootstrap = boot_SE(sample_median, Boston, n=None, B=1000, seed=0)

print(f"An Estimate of the Standard Error of median mu hat by using Bootstrap: {medv_median_se_bootstrap:.5f}")


An Estimate of the Standard Error of median mu hat by using Bootstrap: 0.36945


***
#### (g) Let's compute an estimate for $\hat\mu_{0,1}$, the tenth percentile of medv

In [105]:
medv_10th_percentile = np.percentile(Boston['medv'], 10)
print(f"An Estimate of the 10th Percentile of medv: {medv_10th_percentile:.5f}")

An Estimate of the 10th percentile of medv: 12.75000


***
#### (h) Let's estimate the standard error of $\hat\mu_{0,1}$, the tenth percentile of medv
This is also similar to the problem (f), so all I need to do is just making a new definition of sample_10th_percentile function that returns the tenth percentile of medv for random samples.

After running this code, I found that an estimate of the standard error of $\hat\mu_{0,1}$ by using Bootstrap is 0.50345, which is significantly small.  
The interpretation of this is silmilar to the previous problems, so this means that the 10th percentile of medv in our data set is close to the 10th percentile in the population.

In [109]:
def sample_10th_percentile(D, idx):
    return np.percentile(D['medv'].loc[idx], 10)

# Using boot_SE to estimate the standard error of the 10th percentile.
medv_10th_percentile_se_bootstrap = boot_SE(sample_10th_percentile, Boston, n=None, B=1000, seed=0)

print(f"An estimate of the Standard Error of the 10th Percentile using Bootstrap: {medv_10th_percentile_se_bootstrap:.5f}")


An estimate of the Standard Error of the 10th Percentile using Bootstrap: 0.50345
